<h1>Proyecto Tecnologías de Búsqueda en la Web</h1>
<h3>Integrantes</h3>
<ul><li>Sebastián Aranda</li><li>Felipe Santander</li></ul>

<h2>Librerías</h2>

In [14]:
import gensim
from gensim import corpora
from gensim.corpora import Dictionary, bleicorpus
from gensim.models import ldamodel
from gensim.models import Phrases

from collections import defaultdict

import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

from random import randint

import pyLDAvis
import re
import os
import json

<h2>Creación del Corpus</h2>

In [15]:
corpus_path = "cyberbullying_corpus/"
dataset_path = "Corpus_builder_and_pre/Dataset/"
#corpus_path = "corpus_lda/corpus_lda.lda_c"
#dictionary_path ="corpus_lda/corpus_lda.dict"

In [39]:
#Create bad words list
bad_words = []
with open('badwords','r') as bad_words_file:
    for word in bad_words_file:
        word = word.replace('\n','').decode('unicode_escape').encode('ascii','ignore')
        if word != '':
            bad_words.append(word)
print("Badwords List Size: "+str(len(bad_words)))
        
#Remove keywords in cyberbullying from stopwords list
keywords = ["you", "your", "he", "she", "it"]
stopword_list = [stopword for stopword in stopwords.words('english') if stopword not in keywords]
puncts = ".,:;?!()[]{}~+-\"\'#$%&\/"
digits = "0123456789"

#Create the corpus
print("Creating the corpus...")
mcorpus = []
count = 0
for name in os.listdir(dataset_path):
    if name.endswith('.json'):
        with open(dataset_path+'/'+name) as f:
            count += 1
            op_json = json.loads(f.read())
            try:
                for child in op_json[1]['data']['children']:
                    tokens = [] 
                    
                    #Extrae el comentario
                    comment_text = child['data']['body'].encode('ascii', 'ignore').replace('\n', ' ')
                    #Elimina los links
                    comment_text = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', comment_text)

                    for sym in puncts:
                        comment_text = comment_text.replace(sym," ")
                    for num in digits:
                        comment_text = comment_text.replace(num," ")

                    tokens_comment = [word for word in comment_text.lower().split() if word not in stopword_list]
                    tokens = tokens + tokens_comment

                    try:
                        for child in child['data']['replies']['data']['children']:
                            comment_text = child['data']['body'].encode('ascii', 'ignore').replace('\n', ' ')
                            comment_text = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', comment_text)

                            for sym in puncts:
                                comment_text = comment_text.replace(sym," ")
                            for num in digits:
                                comment_text = comment_text.replace(num," ")

                            tokens_comment = [word for word in comment_text.lower().split() if word not in stopword_list]
                            tokens = tokens + tokens_comment
                    except Exception:
                        pass

                    mcorpus.append(tokens)
            except Exception:
                print('Exception at file:'+str(count)+' has no children')
                pass
        
#Remove words that appear only once
frequency = defaultdict(int)
for doc in mcorpus:
    for token in doc:
        frequency[token] += 1
docs = [[token for token in doc if frequency[token] > 1] for doc in mcorpus]

# Lemmatize all words in documents.
lemmatizer = WordNetLemmatizer()
docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]

# Add bigrams and trigrams to docs (only ones that appear 20 times or more).
bigram = Phrases(docs, min_count=20)
for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            docs[idx].append(token)

#Create and save dictionary
dictionary = corpora.Dictionary(docs)
dictionary.save('tmp/cyberbullying_dictionary.dict') #Save the dictionary

#Convert documents to vectors
corpus = [dictionary.doc2bow(doc) for doc in docs]

#Increasing weight of bad words:
bad_words_ids = [word_id for word_id, word in dictionary.iteritems() if word in bad_words]
ngrams_ids = [word_id for word_id, word in dictionary.iteritems() if "_" in word]

for doc_idx in range(len(corpus)):
    word_id_list = []
    freq_list = []

    for word_id, freq in corpus[doc_idx]:
        word_id_list.append(word_id)
        #if (word_id in bad_words_ids and randint(0,9) > 5):
        if (word_id in bad_words_ids):
            freq_list.append(freq*3)
        else:
            freq_list.append(freq)
    
    new_doc = zip(word_id_list,freq_list)
    corpus[doc_idx] = new_doc

print("Saving Corpus...")
corpora.BleiCorpus.serialize('tmp/cyberbullying_corpus.lda-c', corpus) #Save the corpus


Badwords List Size: 376
Creating the corpus...
Exception at file:5 has no children
Exception at file:115 has no children
Exception at file:293 has no children
Exception at file:390 has no children
Exception at file:507 has no children
Exception at file:694 has no children
Exception at file:963 has no children
Exception at file:1107 has no children
Exception at file:1528 has no children
Exception at file:1617 has no children
Exception at file:1789 has no children
Exception at file:1820 has no children
Exception at file:2185 has no children
Exception at file:2211 has no children
Exception at file:2267 has no children
Exception at file:2382 has no children
Exception at file:2444 has no children
Exception at file:2509 has no children
Exception at file:2657 has no children
Exception at file:2665 has no children
Exception at file:2965 has no children
Exception at file:2976 has no children
Exception at file:2995 has no children
Exception at file:3171 has no children
Exception at file:3686 has

In [17]:
print(corpus[:10])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 2), (11, 1)], [(10, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1)], [(10, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1)], [(34, 1), (35, 1), (36, 1), (37, 1), (38, 1)], [(3, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1)], [(3, 1), (7, 1), (10, 6), (20, 1), (33, 1), (46, 1), (47, 1), (48, 1), (49, 2), (50, 1), (51, 2), (52, 1), (53, 1), (54, 2), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1)], [(1, 1), (10, 2), (33, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 2), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1)], [(3, 1), (10, 3), (20, 1), (39, 1), (69, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1)], [(50, 1), (90, 1), (91, 1), (92, 1), (93, 1)

In [47]:
#Load the dictionary and corpus
if (os.path.exists('tmp/cyberbullying_dictionary.dict') and os.path.exists('tmp/cyberbullying_corpus.lda-c')):
    print('Creating dictionary...')
    %time dictionary = corpora.Dictionary.load('tmp/cyberbullying_dictionary.dict')
    print('Creating corpus...')
    %time corpus = corpora.BleiCorpus('tmp/cyberbullying_corpus.lda-c')
else:
    print("Create the dictionary and corpus first")

Creating dictionary...
CPU times: user 16 ms, sys: 0 ns, total: 16 ms
Wall time: 12.7 ms
Creating corpus...
CPU times: user 120 ms, sys: 4 ms, total: 124 ms
Wall time: 118 ms


In [49]:
#Initialize a model
#print('Creating Tfidf model...')
#%time tfidf = models.TfidfModel(corpus)
#corpus_tfidf = tfidf[corpus]

#Create a LDA Model and save it
%time lda_2 = ldamodel.LdaModel(corpus, num_topics=2, id2word=dictionary)
lda.save('tmp/cyberbullying_ldaModel.lda')

CPU times: user 2min 50s, sys: 64 ms, total: 2min 50s
Wall time: 2min 50s


In [50]:
#Create a LDA Model and save it
%time lda_3 = ldamodel.LdaModel(corpus, num_topics=3, id2word=dictionary)

CPU times: user 2min 47s, sys: 52 ms, total: 2min 47s
Wall time: 2min 46s


In [51]:
#Create a LDA Model and save it
%time lda_4 = ldamodel.LdaModel(corpus, num_topics=4, id2word=dictionary)

CPU times: user 2min 57s, sys: 184 ms, total: 2min 57s
Wall time: 2min 57s


In [52]:
#Create a LDA Model and save it
%time lda_5 = ldamodel.LdaModel(corpus, num_topics=5, id2word=dictionary)

CPU times: user 2min 57s, sys: 148 ms, total: 2min 57s
Wall time: 2min 57s


In [ ]:
#Load LDA Model
lda = models.LdaModel.load('tmp/cyberbullying_ldaModel.lda')

In [41]:
#Print Word Topic Distribution for 100 words
lda.print_topics(2,100)

[(0,
  u'0.047*"you" + 0.031*"your" + 0.018*"fuck" + 0.016*"shit" + 0.016*"it" + 0.014*"fucking" + 0.008*"sex" + 0.007*"she" + 0.007*"get" + 0.007*"he" + 0.006*"one" + 0.006*"time" + 0.006*"roast" + 0.005*"bitch" + 0.005*"know" + 0.005*"u" + 0.005*"face" + 0.005*"picture" + 0.005*"make" + 0.005*"hair" + 0.004*"see" + 0.004*"even" + 0.004*"post" + 0.004*"porn" + 0.004*"pussy" + 0.004*"think" + 0.003*"sure" + 0.003*"never" + 0.003*"good" + 0.003*"say" + 0.003*"suck" + 0.003*"take" + 0.003*"life" + 0.003*"use" + 0.003*"well" + 0.003*"could" + 0.003*"would" + 0.003*"thing" + 0.003*"cock" + 0.003*"oh" + 0.003*"first" + 0.003*"need" + 0.003*"looking" + 0.003*"please" + 0.003*"ever" + 0.003*"faggot" + 0.002*"guy" + 0.002*"rape" + 0.002*"going" + 0.002*"r" + 0.002*"thought" + 0.002*"seen" + 0.002*"deleted" + 0.002*"people" + 0.002*"probably" + 0.002*"right" + 0.002*"really" + 0.002*"pretty" + 0.002*"head" + 0.002*"want" + 0.002*"last" + 0.002*"still" + 0.002*"enough" + 0.002*"as" + 0.002*"bett

In [38]:
#Get top 10 terms of topic 0
for word_id, prob in lda.get_topic_terms(1,10):
    print(dictionary[word_id])

you
year_old
your
fuck
it
love_child
first_time
he
one
get


In [55]:
texts = ["you look like an asshole sitting there in the middle of the room",
        "stop posting stupid shit like that",
         "you look like shit",
         "thank for you help"]

tcorpus = [dictionary.doc2bow(doc.split()) for doc in texts]
for docBow in tcorpus:
    print(lda_2[docBow]) 
print("-----------")
for docBow in tcorpus:
    print(lda_3[docBow]) 
print("-----------")
for docBow in tcorpus:
    print(lda_4[docBow]) 
print("-----------")
for docBow in tcorpus:
    print(lda_5[docBow]) 
print("-----------")

[(0, 0.20508158277607622), (1, 0.79491841722392387)]
[(0, 0.74158652821235715), (1, 0.25841347178764279)]
[(0, 0.36614666211540542), (1, 0.63385333788459464)]
[(0, 0.84074959308012098), (1, 0.15925040691987899)]
-----------
[(0, 0.038192815638495213), (1, 0.91220394550462935), (2, 0.049603238856875523)]
[(0, 0.062898839156764252), (1, 0.5585412666898466), (2, 0.37855989415338914)]
[(0, 0.068804042712018273), (1, 0.86273325297363546), (2, 0.068462704314346284)]
[(0, 0.4616179598475878), (1, 0.0994653893169333), (2, 0.43891665083547893)]
-----------
[(0, 0.52466008448431378), (1, 0.18010846513473605), (2, 0.14258875440776353), (3, 0.15264269597318664)]
[(0, 0.29361461002121242), (1, 0.61508593679948753), (2, 0.047752365065052732), (3, 0.043547088114247311)]
[(0, 0.61297664731687818), (1, 0.28576054819107949), (2, 0.050745146727928653), (3, 0.050517657764113669)]
[(0, 0.065707889722230017), (1, 0.67535695439257237), (2, 0.067469905052357781), (3, 0.19146525083283991)]
-----------
[(0, 0.5